## Biblioteca OR-Tools para la resolución del Problema de Enrutamiento de Vehículos con restricción de carga y de combustible.

### La idea de este Notebook es enseñar el uso que se le puede dar a la biblioteca OR-Tools de Python para la resolución de problema de Optimización, en específico de una versión del Problema de Enrutamiento de Vehículos.
### Primero comenzaremos enseñando cómo se utiliza la biblioteca para la resolución de Problemas de Optimización en general.

### Tomemos como ejemplo de resolución el siguiente problemas:
### Hallar los números $x_{1}$ y $x_{2}$ que maximicen la suma $x_{1} + x_{2}$ tal que $x_{1} \geq 0$ y $x_{2} \geq 0$ y cumplan que:
### $x_{1} + 2x_{2} \leq 4$
### $4x_{1} + 2x_{2} \leq 12$
### $-x_{1} + x_{2} \leq 1$

### Importamos de la biblioteca lo que vamos a utilizar:

In [1]:
from ortools.linear_solver import pywraplp  # Para instalarla: pip install ortools
solver = pywraplp.Solver.CreateSolver('Glop') # Para poder utilizar el linear solver

ModuleNotFoundError: No module named 'ortools'

### Declaración de Variables.
### Esto se hace mediante el método NumVar, que recibe los límites inferior y superior, y el nombre con el que se identificará dicha variable (que puede ser distinto del nombre de la variable donde se guarde la instancia). En el caso de que no existe un límite superior, se utiliza el método infinity. Estos métodos responden al solver que declaramos anteriormente.
### Nota: Además de NumVar también existe IntVar, que solo toma valors enteros

In [ ]:
x_1 = solver.NumVar(0, solver.infinity(), 'x_1')
x_2 = solver.NumVar(0, solver.infinity(), 'x_2')

### Declaración de las Restricciones del Problema.
### Esto se hace mediante del método Add de la instancia de solver que tenemos. La peculiaridad es que la forma de añadir las restricciones se asemeja a su planteamiento matemático. Se realiza de la siguiente forma:

In [ ]:
solver.Add(x_1 + 2*x_2 <= 4)
solver.Add(4*x_1 + 2*x_2 <= 12)
solver.Add((-1)*x_1 + x_2 <= 1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001AC97B90300> >

### Para poder saber el número de restricciones que se tiene podemos utilizar el método NumConstraints

In [ ]:
solver.NumConstraints()

3

### Aplicar la Optimización del Modelo.
### Esto se hace mediante la función Maximize (en caso de que estemos maximizando), o de la función Minimize (en caso contrario). Ambas funciones reciben por argumento la expresión matemática tal y como con las restricciones.
### Luego le pedimos a solver que realiza la optimización con todas las características que estamos pidiendo mediante el método Solve()

In [ ]:
solver.Maximize(x_1 + x_2)
solver.Solve()

0

### Para conocer los valores utilizamos los métodos Objective().Value().

In [ ]:
solver.Objective().Value()

3.333333333333333

### Y para conocer los valores de las variables se usa el método solution_value en las variables que guardamos

In [ ]:
x_1.solution_value()

2.6666666666666665

In [ ]:
x_2.solution_value()

0.6666666666666667

### En este caso el linear solver determinó que no existía un $x_{1}$ o $x_{2} \leq 0$ que pudiera satisfacer dichas restricciones.

## Apliquemos ahora esta librería para el Problema de Enrutamiento de Vehículos.


## Planteamiento del Problema de Enrutamiento de Vehículos:

### Se tiene una flota de vehículos en un centro, que deben satisfacer la demanda de una cierta cantidad de clientes. Cada vehículo tiene un límite de carga que puede llevar y de combustible que puede utilizar para realizar un recorrido, y cada cliente debe recibir cierta cantidad de carga.

### Una forma de afrontar el problema es utilizando optimización lineal. Definimos lo planteado como un grafo dirigido $G = (V, A)$, donde $V = {0,1,...,N}$, el conjunto de vértices, de 1 a N represetan los clientes, siendo 0 el centro del que saldrán los vehículos. A su vez $A$ = {$(i,j): i,j$ pertenece a $V$, $i \neq j$}, es el conjunto de aristas y denotan si existe un conexión entre dos vértices del grafo.
### Cada cliente $i$ ($1 \leq i \leq N$) tiene asociado una demanda de carga $d_{i}$ conocida. Se toma $d_{0} = 0$.
### Cada arista $x_{ij}$ tiene asociado un $c_{ij}$ que representa el gasto de combustible para ir del vértice $i$ al $j$ o viceversa. Se toma como que $c_{ij}$ = $c_{ji}$ (aunque el problema se pudiera aplicar igualmente incluso si esto no se cumpliera). $x_{ij} = 1$ si existe esa arista en el grafo y 0 en caso contrario.
### En el centro hay un cojunto de $K$ Vehículos con iguales características, cada uno con una capacidad $C_{a}$ y cantidad de combustible $C_{o}$. Se toma como que siempre $d_{i} \leq C_{a}$ $\forall$ $i$. Igualmente siempre $C_{ij}$ $\leq$ $C_{o}$ $\forall$ $i$ y $j$.
### Se toma en el problema como que cada cliente será visitado una sola vez, y que los vehículos comienzan y terminan en el centro.
### La idea como modelo de optimización lineal será plantear el problema para que ORTools nos devuelva un grafo que solo contenga aquellas aristas que contemplen las rutas de cada uno de los vehículos, las cuales serán fácil de analizar puesto que solo se pasa por cada vértice una sola vez.
### No se contempla nunca la arista $x_{ij}$ donde $i = j$

## Función Objetivo:

$min \sum_{i = 0}^{n}\sum_{j = 1}^{n} c_{ij}x_{ij} $ $ (1)$ 

Puesto que queremos minimizar el gasto de combustible.

## Restricciones:

$\sum_{i = 0}^{n}\sum_{j = 0}^{n} x_{ij}d_{i} \leq C_{a}$ $\forall$ $S \subset V$ $(2)$

Con esto aseguramos que nunca en un camino se exceda el límite de capacidad de los vehículos.

$\sum_{i = 0}^{n}\sum_{j = 0}^{n} x_{ij}c_{ij} \leq C_{o}$ $\forall$ $S \subset V$ $(3)$

Con esto aseguramos que nunca en un camino se esceda el límite de combustible de algún vehículo.

$\sum_{i = 1}^{n} x_{i0} = K$ $(4)$

Con esto aseguramos que siempre deben salir del Centro los K Vehículos.

$\sum_{j = 1}^{n} x_{0j} = K$ $(5)$


Con esto aseguramos que siempre deben regresar del Centro los K Vehículos.

$\sum_{i = 1}^{n} x_{ij} = 1$ $\forall$ $j$ $(6)$

Con esto aseguramos que siempre se debe entrar a todos los vértices una y solo una vez.

$\sum_{j = 1}^{n} x_{ij} = 1$ $\forall$ $i$ $(7)$

Con esto aseguramos que siempre se debe salir de todos los vértices una y solo una vez.

### Una vez mostrada las ecuaciones, solo nos resta implementar un método que resuelva de forma genérica un problema de Enrutamiento de Vehículos con dischas características.
### ORTools afortunadamente tiene por defecto su propia forma genérica para resolver en específico el problema de enrutamiento de Vehículos, lo cual veremos a continuación.
### Primero importamos la librerías que usaremos de ortools:

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

### Ahora definimos los datos que necesitamos evaluar como modelo de enrutamiento:

In [ ]:
# La cantidad de clientes:
n = 5

# La carga que necesita cada cliente:
# El depósito se le coloca carga 0 por defecto
di = [0, 18, 25, 16, 19, 20]

# La matriz del costo de combustible de ir del punto i al punto j.
xij = [[0, 10, 10, 10, 10, 10],
       [10, 0, 10, 10, 10, 10],
       [10, 10, 0, 10, 10, 10],
       [10, 10, 10, 0, 10, 10],
       [10, 10, 10, 10, 0, 10],
       [10, 10, 10, 10, 10, 0]]

# La cantidad de vehículos
k = 3

# El límite de combustible por vehículo

co = 100

# El límite de carga por vehículo:

ca = 50

### Ahora con ORTools primero definimos los valores generales del modelo a partir de la función pywrapcp.RoutingIndexManager, esta recibe las dimensiones de la matriz objetivo (incluyendo el depósito, lo cual sería n+1), el número de vehículos (k) y la posición del depósito en la matriz (en este caso la posición 0), y esto como tal se lo pasaremos al modelo de enrutamiento, que lo recibe la función pywrapcp.RoutingModel.

In [ ]:
manager = pywrapcp.RoutingIndexManager(n+1, k, 0)

routing = pywrapcp.RoutingModel(manager)

### Ahora pasaremos a definir nuestra función objetivo.
### Para esto vamos a definir un transit_callback, el cual se crea mediante la función RegisterTransitCallback del routing que acabamos de definir, y recibe como argumento una función que dado dos índices le devuelva el valor de costo de las aristas de la matriz.
### Este transit_callback se lo pasaremos a routing mediante su función SetArcCostEvaluatorOfAllVehicles, lo cual definirá los costos en la matriz como nuestra función objetivo.

In [ ]:
def combustible_callback(index_1, index_2):
    # Primero tenemos que transformar los índices de las variables de la ruta guardadas en manager,
    # a los índices de nuestra matriz de costo de combustible.

    index_1 = manager.IndexToNode(index_1)
    index_2 = manager.IndexToNode(index_2)
    return xij[index_1][index_2]

combustible_callback_index = routing.RegisterTransitCallback(combustible_callback)

routing.SetArcCostEvaluatorOfAllVehicles(combustible_callback_index)


### Ahora pasamos a introducir las restricciones.
### Esto se hace mediante la función RegisterUnaryTransitCallback de routing, que de manera idéntica a la función anterior recibe una función de costo a partir de los índices, y que luego se lo pasamos a routing mediante AddDimension, que recibe el callback, la suavidad (que dejaremos en 0), la capacidad de cada vehículo, un bool que indica si comenzar directamente en 0 (en este caso lo dejaremos en True), y el nombre de la restricción.

In [ ]:
def demand_callback(index):
    index = manager.IndexToNode(index)
    return di[index]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

routing.AddDimension(demand_callback_index, 0, ca, True, 'Capacity')


True

### De manera similar añadimos la restricción del combustible, que en este caso reutilizamos el mismo RegisterTransitCallback de la función objetivo (puesto que la restricción de combustible está dada sobre la matriz a optimizar)

In [ ]:

routing.AddDimension(combustible_callback_index, 0, co, True, 'Combustible')

True

### Ahora tomamos de la biblioteca los parámetros de búsqueda que trae y los configuramos:

In [ ]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_parameters.time_limit.FromSeconds(1)

### Y ahora resolvemos el problema

In [ ]:
solution = routing.SolveWithParameters(search_parameters)

### Donde en solution quedan guardadas, por cada iteración, la capacidad actual del vehículo en movimiento, el combustible actual, los movimientos de los vehículos, entre otros datos:

In [ ]:
print(solution)

Assignment(Capacity0 (0) | Capacity1 (0) | Capacity2 (16) | Capacity3 (0) | Capacity4 (20) | Capacity5 (0) | Capacity6 (0) | Capacity7 (0) | Capacity8 (18) | Capacity9 (41) | Capacity10 (39) | Combustible0 (0) | Combustible1 (10) | Combustible2 (20) | Combustible3 (10) | Combustible4 (20) | Combustible5 (10) | Combustible6 (0) | Combustible7 (0) | Combustible8 (20) | Combustible9 (30) | Combustible10 (30) | Nexts0 (1) | Nexts1 (8) | Nexts2 (9) | Nexts3 (2) | Nexts4 (10) | Nexts5 (4) | Nexts6 (3) | Nexts7 (5) | Active0 (1) | Active1 (1) | Active2 (1) | Active3 (1) | Active4 (1) | Active5 (1) | Active6 (1) | Active7 (1) | Vehicles0 (0) | Vehicles1 (0) | Vehicles2 (1) | Vehicles3 (1) | Vehicles4 (2) | Vehicles5 (2) | Vehicles6 (1) | Vehicles7 (2) | Vehicles8 (0) | Vehicles9 (1) | Vehicles10 (2) | (80))


### Sin embargo, la solución de esta forma no está totalmente clara, por tanto vamos a transformar estos datos para que sean más legibles

In [ ]:

print(f'Objetive: {solution.ObjectiveValue()}')
total_distance = 0
total_load = 0

for vehicle_id in range(k):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle: ' + str(vehicle_id) + '\n'
    route_distance = 0
    route_load = 0

    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route_load += di[node_index]
        plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

    plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index), route_load)
    plan_output += 'Distance of the route: {}m\n'.format(route_distance)
    plan_output += 'Load of the route: {}\n'.format(route_load)
    print(plan_output)
    total_distance += route_distance
    total_load += route_load
    
print('Total distance of all routes: {}m\n'.format(total_distance))
print('Total load of all routes: {}\n'.format(total_load))


Objetive: 80
Route for vehicle: 0
 0 Load(0) ->  1 Load(18) ->  0 Load(18)
Distance of the route: 20m
Load of the route: 18

Route for vehicle: 1
 0 Load(0) ->  3 Load(16) ->  2 Load(41) ->  0 Load(41)
Distance of the route: 30m
Load of the route: 41

Route for vehicle: 2
 0 Load(0) ->  5 Load(20) ->  4 Load(39) ->  0 Load(39)
Distance of the route: 30m
Load of the route: 39

Total distance of all routes: 80m

Total load of all routes: 98



### Donde fue confeccionada la respuesta de la forma:
### Ruta del vehículo: i
### posición_actual Load(carga_actual) ->  próxima_posición Load(próxima carga) ->  ...
### Distance of the route: Distancia total de la ruta
### Load of the route: Carga total de la ruta